In [36]:
# imports
from opencadd.structure.core import Structure
from Bio import PDB
from Bio.PDB import PDBIO
import nglview

## Fetch and preprocess PDB
Hamster PKA (5n1f)
* fetch from PDB
* remove waters & small molecules
* save as .pdb file

### Fetch 5n1f

In [2]:
pdb_id = "5N1F"

In [26]:
# Workaround until opencadd is fixed -> TODO manually install biopyhton 1.85 (pip install biopython==1.85)
pdbl = PDB.PDBList()
pdb_filename = pdbl.retrieve_pdb_file(pdb_id, file_format="pdb")

structure = PDB.PDBParser().get_structure(pdb_id, pdb_filename)

Structure exists: '/Users/katharinabuchthal/KinFragLib_PocketEnum/notebooks/n1/pdb5n1f.ent' 


In [16]:
view = nglview.show_biopython(structure)
view.add_representation('licorice',selection="water")
view

NGLWidget()

### remove waters

In [ ]:
preserved_waters = {686, 691, 696}

for chain in structure.get_chains():
    # here we iterate to make it applicable to other structrure but this one has only one chain
    water_to_rm = []
    for res in chain:    
        if res.resname == 'HOH':
            if res.id[1] not in preserved_waters:
                water_to_rm.append(res.id)
            else:
                print(f"Water {res.id[1]} is preserved")

    for water_id in water_to_rm:
        chain.detach_child(water_id)

Water 686 is preserved
Water 691 is preserved
Water 696 is preserved


In [45]:
view = nglview.show_biopython(structure)
view.add_representation('licorice',selection="water")
view

AttributeError: type object 'structureDetails' has no attribute 'level'

We removed all waters but the preserved onces

### remove ligands (8ET, MPD)

In [31]:
partitions = set(res.id[0] for res in structure.get_residues())
partitions

{' ', 'H_8ET', 'H_MPD', 'H_SEP', 'H_TPO', 'W'}

Only 'H_8ET' and 'H_MPD' are ligands, SEP and TPO are modified residues

In [32]:
ligands = {'H_8ET', 'H_MPD'}

In [33]:
for chain in structure.get_chains():
    # here we iterate to make it applicable to other structrure but this one has only one chain
    ligand_res_ids = []
    for res in chain:    
        if res.id[0] in ligands:
            ligand_res_ids.append(res.id)

    for ligand_id in ligand_res_ids:
        chain.detach_child(ligand_id)

In [34]:
view = nglview.show_biopython(structure)
view.add_representation('licorice',selection="water")
view

NGLWidget()

We finished preprocessing of the protein structure -> save it to file 

In [ ]:
io = PDBIO()
io.set_structure(structure)
io.save("5n1f_preprocessed.pdb") # TODO change path

### Binding pocket defintion 
5N1F is hamster PKA thus not in KLIFS => we need to determine the 85 binding pocket residues
=> use pocket def script  
=> make sure to insatll nececarry software (specified in [README](../pocket_definition/README.md))

In [44]:
%run -i '../pocket_definition/pocket_definition.py' '-f=PKA' '-n=Prkaca' '-p=5n1f_preprocessed.pdb'

FileNotFoundError: [Errno 2] No such file or directory: 'clustalo'